In [ ]:
!pip -q install anthropic
!pip -q install langchain google-search-results duckduckgo-search chromadb
!pip -q install PyPDF2 pypdf InstructorEmbedding sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 793.6/793.6 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 95.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.4/396.4 kB 40.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 8.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 966.7/966.7 kB 66.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

In [ ]:
#@title Setting up the Key
import os

os.environ["ANTHROPIC_API_KEY"] = ""


In [ ]:
!pip show langchain

Name: langchain
Version: 0.0.232
Summary: Building applications with LLMs through composability
Home-page: https://www.github.com/hwchase17/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, async-timeout, dataclasses-json, langsmith, numexpr, numpy, openapi-schema-pydantic, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


# Anthropic's Claude-2 with LangChain


We are using the model in the Chat format

In [ ]:
from langchain.chat_models import ChatAnthropic
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import AIMessage, HumanMessage, SystemMessage

In [ ]:
chat = ChatAnthropic(model='claude-2')

In [ ]:
messages = [
    HumanMessage(
        content="How do you compare to gpt-4?"
    )
]
chat(messages)

AIMessage(content=" I don't have a direct comparison to GPT-4 since I'm Claude, an AI assistant created by Anthropic. GPT-4 is an unreleased prototype model developed by Anthropic as well. In general, large language models like GPT continue to improve in fluency, reasoning, and knowledge as they scale up. As an AI assistant focused on being helpful, harmless, and honest, I strive to have natural conversations and provide useful information to users.", additional_kwargs={}, example=False)

## Using it as a basic ChatBot with memory and search

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatAnthropic
from langchain.agents import Tool
from langchain.agents import AgentType
from langchain.tools import DuckDuckGoSearchRun
from langchain.agents import initialize_agent

In [ ]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

llm = ChatAnthropic(model='claude-2',
                 temperature=0)

In [ ]:
search = DuckDuckGoSearchRun()
tools = [
    Tool(
        name = "Current Search",
        func=search.run,
        description="useful for when you need to answer questions about current events or the current state of the world"
    ),
]

In [ ]:
agent_chain = initialize_agent(tools,
                               llm,
                               agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
                               verbose=True,
                               memory=memory)

In [ ]:
agent_chain.run(input="hi, i am sam")



> Entering new AgentExecutor chain...
 ```json
{
    "action": "Final Answer", 
    "action_input": "Hello Sam, nice to meet you! I'm Claude, an AI assistant created by Anthropic to be helpful, harmless, and honest."
}
```

> Finished chain.


"Hello Sam, nice to meet you! I'm Claude, an AI assistant created by Anthropic to be helpful, harmless, and honest."

In [ ]:
agent_chain.run(input="What is Elon Musk's x.ai?")



> Entering new AgentExecutor chain...
 Here is a suggested response using the Current Search tool:

```json
{
    "action": "Current Search",
    "action_input": "Elon Musk's x.ai"
}
```

This will prompt the human to do a current search on "Elon Musk's x.ai" which should provide useful information to help answer their original question. Let me know if you need any clarification on using the response formats!
Observation: Elon Musk has created a new company dedicated to artificial intelligence — and it's called X.AI, as first reported by The Wall Street Journal. The company, which a Nevada filing indicates... CNN — Elon Musk on Wednesday announced the formation of a new company focused on artificial intelligence, after months of teasing plans to build a rival to ChatGPT. The company, called... Musk was one of the founding chairs of OpenAI, along with Altman, OpenAI's CEO. Musk was among a group of investors, including Altman, Greg Brockman, Reid Hoffman, Jessica Livingston, Peter... 

'Elon Musk recently announced the creation of a new company focused on artificial intelligence called X.AI. The company was formed after Musk filed paperwork in Nevada to incorporate an AI firm. X.AI has launched a website and Twitter account. Musk has previously been involved with AI research through organizations like OpenAI, and now appears to be pursuing new AI projects through X.AI.'

In [ ]:
agent_chain.run(input="Who is on the team?")



> Entering new AgentExecutor chain...
 Here is a suggested response using the Current Search tool:

```json
{
  "action": "Current Search",
  "action_input": "Elon Musk X.AI team members"
}
```

This suggests the human look up information on the team members of Elon Musk's new X.AI company using a current internet search. Let me know if you would like me to provide a direct answer instead!
Observation: A state filing listed Musk as its sole director and his personal money manager Jared Birchall, as secretary. Today Musk announced xAI's first 12 employees, including himself. The team consists ... 1:59. AUSTIN, Texas − Elon Musk has formed a new company focused on artificial intelligence. Musk, who also runs Tesla and SpaceX, announced the company's launch on Twitter Wednesday afternoon ... There are 12 members listed on the website as part of the X.AI team, including Musk himself. X.AI employs individuals who previously worked with other AI companies such as OpenAI, Deep Mind ... Elon

'Elon Musk recently announced the creation of a new AI company called X.AI. The initial team consists of 12 members, including Musk himself. Other members come from organizations like OpenAI and DeepMind that specialize in AI research and development. Details on the specific mission and focus of X.AI are still emerging.'

In [ ]:
agent_chain.run(input="What do they plan to do?")



> Entering new AgentExecutor chain...
 ```json
{
    "action": "Current Search", 
    "action_input": "Elon Musk X.AI team plans"
}
```
Observation: But xAI's splash page reveals that the team — 12 people strong, currently — will be led by Musk and veterans of DeepMind, OpenAI, Google Research, Microsoft Research, Tesla and the University... 13 July 2023 Elon Musk is launching a new company called xAI Chesnot/Getty Images What is xAI? xAI is Elon Musk's just-announced artificial intelligence company. Its 12-strong team... A state filing listed Musk as its sole director and his personal money manager Jared Birchall, as secretary. Today Musk announced xAI's first 12 employees, including himself. The team consists ... Elon Musk announces the launch of a new AI company, xAI, which aims to "understand the true nature of the universe". Elon Musk, the owner of Twitter and CEO of Tesla and SpaceX, has launched a ... xAI, Elon Musk's newly formed AI company, has revealed itself with a new web

'Based on my search, it seems Elon Musk recently launched a new AI company called xAI. The initial team consists of 12 members, including veterans from organizations like OpenAI and DeepMind that specialize in AI research. The specific plans and mission of xAI are still emerging, but it appears to be focused on advancing AI capabilities to help understand the nature of the universe.'

## Using Claude-2 for Information Retrieval

In [ ]:
# please put your own zip file of text files in here as for copyright reasons I can't share this one
!wget -q -O elon_xai.zip http:/// etc
!unzip -q elon_xai.zip -d xai_articles

replace xai_articles/1.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace xai_articles/0.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


In [ ]:
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader
from PyPDF2 import PdfReader

from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceInstructEmbeddings

/usr/local/lib/python3.10/dist-packages/InstructorEmbedding/instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


In [ ]:
loader = DirectoryLoader('./xai_articles/', glob="./*.txt", loader_cls=TextLoader)

documents = loader.load()
len(documents)

7

In [ ]:
#splitting the text into
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

In [ ]:
from langchain.embeddings import HuggingFaceInstructEmbeddings

instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl",
                                                      model_kwargs={"device": "cuda"})

load INSTRUCTOR_Transformer
max_seq_length  512


### Making a Chroma DB

In [ ]:
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk
persist_directory = 'db'

## Here is the nmew embeddings being used
embedding = instructor_embeddings

vectordb = Chroma.from_documents(documents=texts,
                                 embedding=embedding,
                                 persist_directory=persist_directory)


In [ ]:
# making out retriever
retriever = vectordb.as_retriever(search_kwargs={"k": 7})

In [ ]:
# create the chain to answer questions
llm = ChatAnthropic(model='claude-2', temperature = 0.0)

qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)

In [ ]:
query = "What did Elon announce?"
llm_response = qa_chain(query)
llm_response["result"]

' Elon Musk announced the launch of a new artificial intelligence company called xAI.'

In [ ]:
query = "Who is on the team?"
llm_response = qa_chain(query)
llm_response["result"]

' I apologize, but I do not actually know the full list of people on the xAI team based on the information provided. The context mentions some notable members like Elon Musk, Igor Babuschkin, Tony Wu, Christian Szegedy, and Greg Yang who previously worked at Google or Microsoft. It also mentions that Dan Hendrycks advises the team. However, a full list of team members is not provided.'

In [ ]:
query = "What is he going to call his GPT model?"
llm_response = qa_chain(query)
llm_response["result"]

' Based on the context provided, it seems Elon Musk intends to call his GPT model \'TruthGPT\'. In an interview from April 2023, Musk said "he wanted to build what he referred to as \'TruthGPT,\' a \'maximum-truth-seeking AI.\'". The announcement for xAI also mentions that "Likely, xAI’s efforts will start there, perhaps taking the form of a text-generating AI that Musk perceives as more \'truthful\' than existing ones". However, without seeing a direct announcement from Musk on the name of his GPT model, I can\'t say for certain what it will be called.'

In [ ]:
query = "Who is going to be ceo?"
llm_response = qa_chain(query)
llm_response["result"]

' Based on the context provided, it appears that Elon Musk will be the CEO of xAI. The announcement states "Our team is led by Elon Musk, CEO of Tesla and SpaceX."'

In [ ]:
query = "Where is the company going to be based?"
llm_response = qa_chain(query)
llm_response["result"]

" I apologize, but I do not have enough information in the provided context to definitively say where xAI will be based. The context mentions that xAI is recruiting engineers and researchers in the Bay Area, and most AI development has been concentrated in Silicon Valley. It also says that Musk recently traveled to China and discussed AI with government officials there. However, there is no explicit statement about where xAI's headquarters or main office will be located."

In [ ]:
query = "What did Musk say about China and AI?"
llm_response = qa_chain(query)
print(llm_response["result"])

 Based on the context provided, Elon Musk said that China is interested in working cooperatively on AI regulation. Specifically, he said:

"China is definitely interested in working in a cooperative international framework for AI regulation," Musk said.

He added that he advocated for AI regulations and oversight in his meetings in China.

Beyond that, I don't know exactly what else Musk said about China and AI. The context talks about Musk's recent trip to China and meetings with government officials there, as well as China's recent interim measures on AI, but does not provide direct quotes from Musk on anything else related to China and AI.
